First, we activate the right Julia environment and add missing packages.

Afterwards, Julia will have to precompile the environment again. This may take some time ...

In [ ]:
using Pkg
Pkg.activate("./")
Pkg.add("Zygote")
Pkg.add("Plots")

In [ ]:
using VeryDiff, VNNLib, Zygote, Plots, LinearAlgebra

# Gradient-based Attacks

We define the *fast gradient sign method* (FGSM) and *projected gradient descent* (PGD).

In [ ]:
function fgsm(net1, net2, x, lbs, ubs, loss_fn; step_size=0.1)
    y, ∇ = withgradient(x -> loss_fn(net1(x), net2(x)), x)
    return clamp.(x .- ∇[1] .* step_size, lbs, ubs)
end

function pgd(net1, net2, x, lbs, ubs, loss_fn; step_size=0.01, iterations=100)
    best_y = Inf
    best_x = x
    
    for i in 1:iterations
        y, ∇ = withgradient(x -> loss_fn(net1(x), net2(x)), x)

        if y < best_y
            best_y = y
            best_x = x
        end

        x = clamp.(x .- ∇[1] .* step_size, lbs, ubs)
    end

    return best_x
end 

Then, we define a loss function to maximize the violation of $\epsilon$-equivalence.

In [ ]:
ϵ_equiv_loss = (y1, y2) -> .-maximum(abs.(y1 .- y2))

And we can try to find concrete images that violate the $\epsilon$-equivalence property for concrete networks.

In [ ]:
# net1 = load_network("/data/mnist_relu_3_100.onnx");
# net2 = load_network("/data/mnist_relu_3_100_pruned5.onnx");
net1 = load_network("./test/examples/nets/mnist_relu_3_100.onnx")
net2 = load_network("./test/examples/nets/mnist_relu_3_100_pruned5.onnx")

Select an input set, we want to test.

In [ ]:
# try one of those
f, n_in, n_out = get_ast("./test/examples/specs/mnist_7_local_15.vnnlib")
# f, n_in, n_out = get_ast("/data/mnist_90_global_4.vnnlib");
# f, n_in, n_out = get_ast("/data/mnist_9_global_3.vnnlib");

And try to find an input with a maximal absolute difference $> 1$.

In [ ]:
x0 = nothing
x1 = nothing
for (bounds, matrix, bias, num) in f
   lbs = bounds[1:n_in,1]
   ubs = bounds[1:n_in,2]
   x0 = 0.5 .* (lbs .+ ubs)
   x1 = pgd(net1, net2, x0, lbs, ubs, ϵ_equiv_loss)


   @show net1(x0)
   @show net2(x0)
   @show ϵ_equiv_loss(net1(x0), net2(x0))
   @show ϵ_equiv_loss(net1(x1), net2(x1))
end

In [ ]:
p1 = heatmap(reshape(x0, 28, 28)')
p2 = heatmap(reshape(x1, 28, 28)')

plot(p1, p2)

# Zonotopes Introduction

Zonotopes can be created via their generator matrix and center.

We can also plot 2D Zonotopes.

In [ ]:
include("util.jl")

#            G     c
z = Zonotope(I(2), zeros(2), nothing)

plot(z, alpha=0.5, framestyle=:origin, aspect_ratio=:equal)

Can you create a Zonotope that represents a (not necessarily regular) 2-dimensional hexagon or octagon?

In [ ]:
z = Zonotope(Matrix([1.0 0; 1.0 0]), zeros(2), nothing)

plot(z, alpha=0.5, framestyle=:origin, aspect_ratio=:equal)

In [ ]:
x = range(-1, 1, length=1000)
y = max.(x, 0)

λ = 0.5
u = λ * x .+ λ
l = λ * x
m = 0.5 * (u + l)
# ̂y = λx + λϵ
linewidth = 1.5

plot(x, y, label="ReLU(x)", lc=:blue, lw=linewidth)
# plot!(x, l, label="lower(x)", lc=:blue)
# plot!(x, u , label="upper(x)", lc=:green)
plot!(x, m , label="Vertical Splitting", lc=:red, lw=linewidth)

#            G     c
z = Zonotope(Matrix([1.0 0; λ -λ*λ]), [0, 0.5*λ], nothing)
plot!(z, alpha=0.3, label="Zonotope", framestyle=:origin, aspect_ratio=:equal, lw=linewidth)

# z = Zonotope(Matrix([1.0 0; λ -λ*λ*λ]), [0, 0.5*λ*(λ + 1.0)], nothing)
# plot!(z, alpha=0.3, label="Zonotope Z1", framestyle=:origin, aspect_ratio=:equal, lw=linewidth)

# z = Zonotope(Matrix([1.0 0; λ -λ*λ*λ]), [0, 0.5*λ*λ], nothing)
# plot!(z, alpha=0.3, label="Zonotope Z2", framestyle=:origin, aspect_ratio=:equal, lw=linewidth)

# savefig("vertical_splitting.png")

In [ ]:
function affine_transform(L::Dense, Z :: Zonotope)
    return begin
        G = L.W * Z.G
        c = L.W * Z.c .+ L.b
        return Zonotope(G, c, Z.influence)
    end
end

function relu_transform_alt(Z :: Zonotope)
    return begin
        bounds = zono_bounds(Z)
        lower = bounds[:, 1]
        upper = bounds[:, 2]
        
        α = clamp.(upper./(upper.-lower), 0.0, 1.0)
        λ = ifelse.(upper.<=0.0, 0.0, ifelse.(lower.>=0.0, 1.0, α))
        crossing = lower.<0.0 .&& upper.>0.0
        γ = 0.5 .* max.(-λ .* lower, 0.0, ((-).(1.0,λ)).*upper)  # Computed offset (-λl/2)
        
        ĉ = λ .* Z.c .+ crossing.*γ

        row_count = size(Z.G, 1)
        Ĝ = zeros(Float64, row_count, size(Z.G,2)+1)

        Ĝ[:,1:size(Z.G,2)] .= Z.G
        Ĝ[crossing,size(Z.G,2)+1] .= 1.0

        Ĝ[:,1:size(Z.G,2)] .*= λ
        Ĝ[:,size(Z.G,2)+1] .*= abs.(γ)

        return Zonotope(Ĝ, ĉ, Z.influence)
    end
end

function relu_transform(Z :: Zonotope)
    return begin
        bounds = zono_bounds(Z)
        lower = bounds[:, 1]
        upper = bounds[:, 2]
        
        α = clamp.(upper./(upper.-lower), 0.0, 1.0)
        λ = ifelse.(upper.<=0.0, 0.0, ifelse.(lower.>=0.0, 1.0, α))
        crossing = lower.<0.0 .&& upper.>0.0
        γ = 0.5 .* max.(-λ .* lower, 0.0, ((-).(1.0,λ)).*upper)  # Computed offset (-λl/2)
        
        ĉ = λ .* Z.c .+ crossing.*γ

        row_count = size(Z.G, 1)
        Ĝ = zeros(Float64, row_count, size(Z.G,2)+count(crossing))

        Ĝ[:,1:size(Z.G,2)] .= Z.G
        Ĝ[crossing,size(Z.G,2)+1:end] .=  (@view I(row_count)[crossing, crossing])

        Ĝ[:,1:size(Z.G,2)] .*= λ
        Ĝ[:,size(Z.G,2)+1:end] .*= abs.(γ)

        return Zonotope(Ĝ, ĉ, Z.influence)
    end
end

In [ ]:
Z₀ = Zonotope([2.0 0.0; 0.0 -1.0], [0, 1], nothing)

f₁ = Dense([-1.0 2.0; 1.0 0.0], [0.0, 1]) #Dense([1.0 1.0; 1.0 2.0], [1.0, 1.0])
f₂ = Dense([3.0 1.0; 1.0 1.0], [1.0, 0.0]) #Dense([1.0 1.0; 2.0 1.0], [1.0, 1.0])
f₃ = Dense([1.0 -1.0; -5.0 1.0], [1.0, 2.0]) #Dense([1.0 2.0; 1.0 1.0], [1.0, 1.0])
f₄ = Dense([-3.0 1.0; -2.0 5.5], [1.0, -1.0]) #Dense([2.0 1.0; 1.0 1.0], [1.0, 1.0])

layers = [f₁, f₂, f₃, f₄]

Z = Z₀
Z̃ = Z

for f in layers
    Z = relu_transform(affine_transform(f, Z))
    Z̃ = relu_transform_alt(affine_transform(f, Z̃))
    bounds₁ = zono_bounds(Z)
    bounds₂ = zono_bounds(Z̃)
    println("Bounds₁: $bounds₁")
    println("Bounds₂: $bounds₂")
end


# Zonotope Propagation

We can define our example network as a list of layers.

In [ ]:
W1 = [1 1; 1 -1.]
b1 = zeros(2)
W2 = [1 1; 1 -1.]
b2 = [-0.5, 0]
W3 = [-1 0; 1 1.]
b3 = zeros(2)

layers = [Dense(W1, b1), ReLU(), Dense(W2, b2), ReLU(), Dense(W3, b3)]

nn = Network(layers)

Now we can define our input zonotope and see how it propagates through the layers of the network.

In [ ]:
# needed for VeryDiff propagation
prop_state = PropState(false);

# zonotope defined by generator matrix and center (and influence vector for splitting heuristic later on)
z = Zonotope(I(2), zeros(2), zeros(2)')

In [ ]:
plot(z, alpha=0.5, framestyle=:origin, xlabel="x₁", ylabel="x₂", aspect_ratio=:equal)

In [ ]:
ẑ₁ = layers[1](z, prop_state)

@show ẑ₁

ys = sample_nn(nn, z, max_layer=1, n_samples=1000)

plot(ẑ₁, alpha=0.5, framestyle=:origin, xlabel="n̂₁", ylabel="n̂₂", aspect_ratio=:equal, label="Zonotope")
scatter!(ys[1,:], ys[2,:], label="NN(x)")

In [ ]:
z₁ = layers[2](ẑ₁, prop_state)

@show z₁

ys = sample_nn(nn, z, max_layer=2, n_samples=1000)

plot(z₁, alpha=0.5, framestyle=:origin, xlabel="n₁", ylabel="n₂", aspect_ratio=:equal, label="Zonotope")
scatter!(ys[1,:], ys[2,:], label="NN(x)")

In [ ]:
@show layers[3]

In [ ]:
ẑ₂ = layers[3](z₁, prop_state)

@show ẑ₂

ys = sample_nn(nn, z, max_layer=3, n_samples=1000)

plot(ẑ₂, alpha=0.5, framestyle=:origin, xlabel="n̂₃", ylabel="n̂₄", aspect_ratio=:equal, label="Zonotope")
scatter!(ys[1,:], ys[2,:], label="NN(x)")

In [ ]:
@show layers[4]

In [ ]:
z₂ = layers[4](ẑ₂, prop_state)

@show z₂

ys = sample_nn(nn, z, max_layer=4, n_samples=1000)

plot(z₂, alpha=0.5, framestyle=:origin, xlabel="n₃", ylabel="n₄", aspect_ratio=:equal, label="Zonotope")
scatter!(ys[1,:], ys[2,:], label="NN(x)")

In [ ]:
@show layers[5]

In [ ]:
ẑ₃ = layers[5](z₂, prop_state)

@show ẑ₃

ys = sample_nn(nn, z, max_layer=5, n_samples=1000)

plot(ẑ₃, alpha=0.5, framestyle=:origin, xlabel="y₁", ylabel="y₂", aspect_ratio=:equal, label="Zonotope")
scatter!(ys[1,:], ys[2,:], label="NN(x)")

# Refinement via Input Splitting

Can you find a partition of the input s.t. you can prove that $y_2 > -2$?

In [ ]:
# we can also define Zonotopes directly by the lower and upper bounds of the variables
z = Zonotope([-1., -1.], [1., 1.])  # original input set

# splits
zs = [
    Zonotope([-1., -1.], [0., 1])
    Zonotope([ 0., -1.], [1., 1])
]

You can visually check, if your splits are a partition of the initial input set.

In [ ]:
p = plot(z, alpha=0.5, framestyle=:origin, xlabel="y₁", ylabel="y₂", aspect_ratio=:equal, label="Zonotope")

for (i, zi) in enumerate(zs)
    plot!(zi, alpha=0.5, label="z$i")
end

p

In [ ]:
zs_out = [nn(zi, prop_state) for zi in zs]

In [ ]:
ẑ₃ = nn(z, prop_state)
p = plot(ẑ₃, alpha=0.5, framestyle=:origin, xlabel="y₁", ylabel="y₂", aspect_ratio=:equal, label="Zonotope")

for (i, zi) in enumerate(zs_out)
    plot!(zi, alpha=0.5, label="z$i")
end

p

# Equivalence Verification

Now we define another network with very similar weight matrices to our original network.

We want to prove that their absolute output difference is $\leq 1$ for our input set.

In [ ]:
W1a = [1 1.1; 1 -0.9]
b1a = zeros(2)
W2a = [1.1 1; 0.9 -1]
b2a = [-0.4, 0]
W3a = [-1 0; 1 1.1]
b3a = zeros(2)

layersa = [Dense(W1a, b1a), ReLU(), Dense(W2a, b2a), ReLU(), Dense(W3a, b3a)]

nn1 = nn;
nn2 = Network(layersa);

If we just propagate our input zonotope individually through each network, this is what we get:

In [ ]:
z = Zonotope(-ones(2), ones(2))
prop_state = PropState(true)

z1 = nn1(z, prop_state)
z2 = nn2(z, prop_state);

In [ ]:
plot(z1, label="z1", alpha=0.5, framestyle=:origin, xlabel="y₁", ylabel="y₂")
plot!(z2, label="z2", alpha=0.5)

Can you write a function that computes a zonotope overapproximating the possible values of $NN_1(z) - NN_2(z)$?

In [ ]:
function difference_bounds(net1, net2, z)
    # TODO: complete this method
end

You can test the correctness of your function using sampled points.

In [ ]:
zΔ = difference_bounds(nn1, nn2, z)

In [ ]:
xs = sample_zono(z, n_samples=1000)

ys1 = hcat([nn1(Vector(xi)) for xi in eachcol(xs)]...)
ys2 = hcat([nn2(Vector(xi)) for xi in eachcol(xs)]...)

plot(zΔ, alpha=0.5, framestyle=:origin, label="zΔ")
plot!(zΔ2, alpha=0.5, label="zΔ2")
scatter!(ys1[1,:] .- ys2[1,:], ys1[2,:] .- ys2[2,:], label="sampled")

## Functions for just the Zonotope Difference

In [ ]:
function difference_zonotope(z1, z2, n_in)
    # TODO: complete this function

    # This would be an error, because they don't care about separate generators
    G = z1.G .- z2.G
    c = z1.c .- z2.c

    return Zonotope(G, c)
end

In [ ]:
function difference_zonotope2(z1, z2, n_in)
    # this would work in this instance (when we always have more than n_in generators
    G_shared = z1.G[:,1:n_in] .- z2.G[:,1:n_in]
    G = [G_shared z1.G[:,n_in+1:end] .-z2.G[:,n_in+1:end]]

    c = z1.c .- z2.c

    return Zonotope(G, c)
end

# Differential Verification Layer by Layer

We define a structure that captures all three networks that are involved:
- $NN_1$
- $NN_2$
- and the difference network

In [ ]:
nn_diff = GeminiNetwork(nn1, nn2);

We also have an input zonotope and set the initial difference zonotope to $0$ (as both networks are evaluated on the same inputs).

In [ ]:
∂z = Zonotope(zero(z.G), zero(z.c), nothing)
zΔ = DiffZonotope(z, deepcopy(z), ∂z, 0, 0, 0)

prop_state = PropState(true);

We can now watch how the difference zonotope for the naive approach grows vs the zonotope for the differential verification.

In [ ]:
plot(zΔ.Z₁, label="z1", framestyle=:origin, alpha=0.5, title="input")
plot!(zΔ.Z₂, label="z2", alpha=0.5)
plot!(zΔ.∂Z, label="∂z", alpha=0.5)

In [ ]:
zΔ1 = VeryDiff.propagate_diff_layer((nn_diff.network1.layers[1], nn_diff.diff_network.layers[1], nn_diff.network2.layers[1]), zΔ, prop_state)

plot(zΔ1.Z₁, label="z1", framestyle=:origin, alpha=0.5, title="Dense1")
plot!(zΔ1.Z₂, label="z2", alpha=0.5)
plot!(zΔ1.∂Z, label="∂z", alpha=0.5)
# naive difference
z_diff = difference_zonotope2(zΔ1.Z₁, zΔ1.Z₂, 2)
plot!(z_diff, label="naive", alpha=0.5)

In [ ]:
i = 2
zΔ2 = VeryDiff.propagate_diff_layer((nn_diff.network1.layers[i], nn_diff.diff_network.layers[i], nn_diff.network2.layers[i]), zΔ1, prop_state)

plot(zΔ2.Z₁, label="z1", framestyle=:origin, alpha=0.5, title="ReLU1")
plot!(zΔ2.Z₂, label="z2", alpha=0.5)
plot!(zΔ2.∂Z, label="∂z", alpha=0.5)

z_diff = difference_zonotope2(zΔ2.Z₁, zΔ2.Z₂, 2)
plot!(z_diff, label="naive", alpha=0.5)

In [ ]:
i = 3
zΔ3 = VeryDiff.propagate_diff_layer((nn_diff.network1.layers[i], nn_diff.diff_network.layers[i], nn_diff.network2.layers[i]), zΔ2, prop_state)

plot(zΔ3.Z₁, label="z1", framestyle=:origin, alpha=0.5, title="Dense2")
plot!(zΔ3.Z₂, label="z2", alpha=0.5)
plot!(zΔ3.∂Z, label="∂z", alpha=0.5)

z_diff = difference_zonotope2(zΔ3.Z₁, zΔ3.Z₂, 2)
plot!(z_diff, label="naive", alpha=0.5)

In [ ]:
i = 4
zΔ4 = VeryDiff.propagate_diff_layer((nn_diff.network1.layers[i], nn_diff.diff_network.layers[i], nn_diff.network2.layers[i]), zΔ3, prop_state)

plot(zΔ4.Z₁, label="z1", framestyle=:origin, alpha=0.5, title="ReLU2")
plot!(zΔ4.Z₂, label="z2", alpha=0.5)
plot!(zΔ4.∂Z, label="∂z", alpha=0.5)

z_diff = difference_zonotope2(zΔ4.Z₁, zΔ4.Z₂, 2)
plot!(z_diff, label="naive", alpha=0.5)

In [ ]:
i = 5
zΔ5 = VeryDiff.propagate_diff_layer((nn_diff.network1.layers[i], nn_diff.diff_network.layers[i], nn_diff.network2.layers[i]), zΔ4, prop_state)

plot(zΔ5.Z₁, label="z1", framestyle=:origin, alpha=0.5, title="Dense3")
plot!(zΔ5.Z₂, label="z2", alpha=0.5)
plot!(zΔ5.∂Z, label="∂z", alpha=0.5)

z_diff = difference_zonotope2(zΔ5.Z₁, zΔ5.Z₂, 2)
plot!(z_diff, label="naive", alpha=0.5)

# Larger Examples

We can now try `VeryDiff` (and the state-of-the-art non-differential verifier $\alpha$-$\beta$-CROWN) on larger MNIST networks.

In [ ]:
net1 = load_network("/data/mnist_relu_3_100.onnx");
net2 = load_network("/data/mnist_relu_3_100_pruned5.onnx");

Choose a property to verify

In [ ]:
f, n_in, n_out = get_ast("/data/mnist_9_global_3.vnnlib");
#f, n_in, n_out = get_ast("/software/networks/mnist_67_global_6.vnnlib");
#f, n_in, n_out = get_ast("/data/mnist_90_global_4.vnnlib");
#f, n_in, n_out = get_ast("/data/mnist_55_local_24.vnnlib");
#f, n_in, n_out = get_ast("/data/mnist_64_local_15.vnnlib");

And see how `VeryDiff` performs.

You can also go back to the section about adversarial attacks: Are there any contradictions? Are there cases, where `VeryDiff` can find counterexamples that PGD could not find?

In [ ]:
for (bounds, matrix, bias, num) in f
   prop = get_epsilon_property(1.)
   verify_network(net1, net2, bounds[1:n_in,:], prop, epsilon_split_heuristic, timeout=120.)
end

To run $\alpha$-$\beta$-CROWN, you need to start a terminal and enter
```bash
$ cd /software/abCROWN
$ conda activate alpha-beta-crown
$ python complete_verifier/abcrown.py --onnx_path /data/product_mnist_relu_3_100_mnist_relu_3_100_pruned5_eps.onnx --vnnlib_path /data/product_mnist_67_global_6_eps_1.0.vnnlib --device cpu --config /data/mnistfc_modifed.yaml 
```
you can adjust `--vnnlib_path` as you wish. 

Just make sure to always choose the `product_mnist...` property (which encodes the naive difference).

How does it compare to `VeryDiff` (on UNSAT/SAT instances)?